In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:

from solar_power.environment_prediction_model import SolarPredictionModelExtended 
from solar_power.solar_prediction_model import FeatureSolarPredictionModel
from electrical_load.load_prediction_model import BuildingEnergyModel

# * 모델 선언
# * 랜덤 시드 값을 모델 간 동일하게 42로 설정
# * 테스트 데이터 개수는 모델 간 동일하게 12 설정, 12/62 = 19.35%

random_state = 42
test_size = 12 

# * 기상 데이터 to 환경 감시 데이터 예측 모델
env_pred_model = SolarPredictionModelExtended("./solar_power/merged_result.csv", random_state=random_state, test_size=test_size)
env_pred_model.load_data()
env_pred_model.hyperparameter_tuning()
env_pred_model.train_model()

# * 환경 감시 데이터 to 태양광 발전량 예측 모델
solar_pred_model = FeatureSolarPredictionModel("./solar_power/merged_result.csv", random_state=random_state, test_size=test_size)
solar_pred_model.load_data()
solar_pred_model.feature_selection()
solar_pred_model.train_model()

# * 건물 전기 부하 예측 모델
energy_pred_model = BuildingEnergyModel("./electrical_load/merged_data.csv", random_state=random_state, test_size=test_size)
energy_pred_model.load_data()
energy_pred_model.train_model()

In [5]:

# * 태양광 발전량 예측 모델에 입력할 정규화된 입력 데이터
# * test_x 는 1시간 단위의 데이터
# * test_date 는 예측할 시간의 날짜

import numpy as np
rand_int = np.random.randint(0, len(env_pred_model.X_test))

test_x = env_pred_model.X_test.iloc[rand_int:rand_int+1, :]
test_index = test_x.index[0]
test_date = env_pred_model.data.loc[test_index, 'day']

test_x


,강수확률,일최저기온,하늘상태,일최고기온,습도,풍향,풍속,time
723,0.3,0.9,1.0,0.714286,0.75,0.975528,0.354839,0.565217


In [6]:

# * 1시간 단위의 데이터와 같은 날짜의 데이터 24개를 합쳐 same_day_X_test 생성

same_day_data = env_pred_model.data[env_pred_model.data['day'] == test_date]
same_day_data_indexes = same_day_data.index
same_day_X_test = env_pred_model.X_test.loc[env_pred_model.X_test.index.isin(same_day_data_indexes)]

same_day_X_test

,강수확률,일최저기온,하늘상태,일최고기온,습도,풍향,풍속,time
720,0.3,0.9,1.0,0.714286,0.750000,0.989074,0.370968,0.500000
721,0.3,0.9,1.0,0.714286,0.750000,0.972759,0.370968,0.521739
722,0.3,0.9,1.0,0.714286,0.750000,0.972759,0.370968,0.543478
723,0.3,0.9,1.0,0.714286,0.750000,0.975528,0.354839,0.565217
724,0.3,0.9,1.0,0.714286,0.750000,0.978152,0.370968,0.586957
725,0.3,0.9,1.0,0.714286,0.833333,0.982963,0.370968,0.608696
726,0.6,0.9,1.0,0.714286,0.833333,0.972759,0.435484,0.630435
727,0.6,0.9,1.0,0.714286,0.833333,0.963592,0.548387,0.652174
728,0.6,0.9,1.0,0.714286,0.833333,0.953154,0.596774,0.673913
729,0.6,0.9,1.0,0.714286,0.833333,0.941474,0.596774,0.695652


# 태양광 발전량 예측

In [7]:

# * 예측한 태양광 발전량 데이터

res_solar = env_pred_model.predict(same_day_X_test) # 기상 데이터 -> 환경감시 데이터
res_solar = solar_pred_model.predict(res_solar) # 환경감시 데이터 -> 태양광 발전량 데이터
res_solar

array([ -4.64999725,  -4.73836412,  -4.73836412,  -5.23782566,
        -4.64999725,   7.47409053,   2.96811202, 339.31413348,
       643.43166553, 715.32871273, 701.83723191, 703.48159469,
       748.51764918, 682.05851251, 678.10837997, 490.28704703,
       442.92970219, 316.35271242,  -1.69884978,   5.07918022,
        -1.85851564,  -2.96335657,   3.7326413 ,   5.39781704])

In [8]:
# * 실제 태양광 발전량

same_day_y_test = solar_pred_model.y_test.loc[solar_pred_model.y_test.index.isin(same_day_data_indexes)]
same_day_y_test

,축구장,학생회관,중앙창고,학사과정,다산빌딩,시설관리동,대학C동,동물실험동,중앙도서관,LG도서관,신재생에너지동,삼성환경동,중앙연구기기센터,산업협력관,기숙사 B동
720,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
721,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
722,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
723,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
724,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
725,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
726,0.0,0.0,0.0,0.0,-0.2,0.0,0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,-0.0
727,3.2,2.0,1.0,0.0,1.0,4.2,0.8,1.9,1.4,1.7,3.4,2.6,4.7,5.2,2.0
728,11.3,6.7,3.9,3.6,4.3,15.1,2.2,5.4,4.3,5.8,9.9,8.1,14.7,15.0,6.6
729,13.7,7.3,4.3,3.9,5.6,17.9,2.5,6.7,4.9,6.7,11.4,9.8,17.9,19.5,7.7


# 전기 부하량 예측

In [9]:
# * 부하량 예측 모델에 입력할 정규화된 입력 데이터

same_day_X_test = energy_pred_model.X_test.loc[energy_pred_model.X_test.index.isin(same_day_data_indexes)]
same_day_X_test

,time,1시간기온,1시간강수량,일최고기온,일최저기온,weekday_Friday,weekday_Monday,weekday_Saturday,weekday_Sunday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday
720,0,27.0,0.0,32.0,25.0,False,False,False,True,False,False,False
721,1,27.0,0.0,32.0,25.0,False,False,False,True,False,False,False
722,2,27.0,0.0,32.0,25.0,False,False,False,True,False,False,False
723,3,27.0,0.0,32.0,25.0,False,False,False,True,False,False,False
724,4,26.0,0.0,32.0,25.0,False,False,False,True,False,False,False
725,5,26.0,0.0,32.0,25.0,False,False,False,True,False,False,False
726,6,26.0,4.0,32.0,25.0,False,False,False,True,False,False,False
727,7,27.0,3.0,32.0,25.0,False,False,False,True,False,False,False
728,8,27.0,4.0,32.0,25.0,False,False,False,True,False,False,False
729,9,27.0,0.0,32.0,25.0,False,False,False,True,False,False,False


In [10]:
# * 예측한 전기 부하량

res_load = energy_pred_model.predict(same_day_X_test)
res_load

,중앙P/P_석사,전기전자컴퓨터공학동_석사,신소재공학_석사,생명과학동_석사,기계공학동_석사,LG도서관_석사,창업B동_석사,금호관_석사,"냉각수순환펌프1,2,3_석사","냉각수순환펌프4,5,6_석사",...,대학B동(E)_학사,대학기숙사A동(E)_학사,학사P/P(E)_학사,제2학생회관(E)_학사,교원아파트(E)_학사,대학C동(E)_학사,중앙연구기기센터(E)_학사,대학A동(E)_학사,다산빌딩(E)_학사,산학협력연구동(E)_학사
0,158.8,444.2,444.6,468.6,117.8,252.6,32.1,54.3,0.0,0.0,...,30.9,40.0,40.2,27.3,23.1,18.1,49.0,17.1,50.0,17.6
1,159.3,440.3,440.6,452.6,117.0,243.2,29.0,54.5,0.0,0.0,...,30.4,39.2,38.2,26.4,24.0,19.5,48.5,16.6,46.8,18.0
2,154.5,437.1,440.1,453.9,114.1,238.2,34.2,54.7,0.0,0.0,...,27.2,32.4,38.7,24.7,22.2,18.9,48.6,14.2,42.8,18.4
3,155.0,430.9,433.8,490.0,111.9,233.9,31.5,54.4,0.0,0.0,...,26.7,33.2,41.3,24.3,20.4,22.2,48.8,15.7,42.5,18.2
4,144.4,428.0,414.5,458.9,105.1,243.9,32.1,53.8,0.0,0.0,...,28.1,30.9,40.5,27.7,17.6,17.4,48.7,14.3,37.7,17.4
5,142.1,428.1,426.4,450.3,107.9,230.0,35.7,54.1,0.0,0.0,...,27.5,26.1,35.2,27.2,21.4,17.4,48.1,15.6,38.2,17.9
6,118.0,426.1,424.5,470.6,114.3,241.6,30.8,54.8,2.1,2.9,...,30.1,25.0,27.5,33.2,26.7,17.9,48.8,17.0,40.9,18.9
7,107.3,427.8,416.9,481.8,117.0,251.4,32.4,54.9,10.5,14.3,...,31.2,31.3,29.9,28.8,32.2,19.4,50.5,19.9,41.9,19.1
8,82.4,422.6,396.1,447.1,122.4,267.5,28.7,52.7,0.0,0.0,...,25.1,36.9,26.0,24.9,30.5,27.2,47.5,14.9,44.2,18.3
9,82.7,424.6,396.7,453.0,127.5,273.9,34.0,53.9,0.0,0.0,...,25.9,33.3,25.6,25.6,31.5,24.2,48.2,15.0,46.5,18.2


In [11]:
# * 실제 전기 부하량

same_day_y_test = energy_pred_model.y_test.loc[energy_pred_model.y_test.index.isin(same_day_data_indexes)]
same_day_y_test

,중앙P/P_석사,전기전자컴퓨터공학동_석사,신소재공학_석사,생명과학동_석사,기계공학동_석사,LG도서관_석사,창업B동_석사,금호관_석사,"냉각수순환펌프1,2,3_석사","냉각수순환펌프4,5,6_석사",...,대학B동(E)_학사,대학기숙사A동(E)_학사,학사P/P(E)_학사,제2학생회관(E)_학사,교원아파트(E)_학사,대학C동(E)_학사,중앙연구기기센터(E)_학사,대학A동(E)_학사,다산빌딩(E)_학사,산학협력연구동(E)_학사
720,157.2,453.5,423.0,466.5,114.1,278.6,45.9,59.5,0.0,0.0,...,37.5,39.3,40.7,30.4,14.9,30.3,49.0,27.1,49.4,17.4
721,144.1,428.3,414.7,453.2,106.3,258.7,33.7,60.4,0.0,0.0,...,35.3,40.8,37.8,22.8,14.7,15.8,47.6,15.0,46.7,18.1
722,130.7,434.5,414.3,451.9,111.3,254.4,20.6,56.1,0.0,0.0,...,25.3,33.1,40.6,30.8,17.8,16.7,47.9,20.1,45.0,17.5
723,126.9,421.3,412.1,450.4,108.0,267.7,29.0,55.5,0.0,0.0,...,25.4,38.8,51.6,23.4,14.1,16.9,56.4,14.5,40.3,17.5
724,132.0,420.7,411.8,490.9,106.4,257.7,30.1,70.0,0.0,0.0,...,32.0,30.4,38.7,20.9,14.4,15.7,47.6,13.6,33.2,17.5
725,121.0,431.3,412.3,451.9,101.8,249.3,29.3,56.9,0.0,0.0,...,25.9,19.7,32.0,21.1,27.5,15.5,50.4,15.7,31.5,16.9
726,94.1,434.3,404.1,452.3,104.0,249.4,27.0,56.7,0.0,0.0,...,24.2,32.1,20.3,25.9,26.8,15.3,48.8,13.3,31.1,17.2
727,89.6,432.1,413.9,449.0,110.6,255.6,42.0,63.3,0.0,0.0,...,25.5,32.1,21.9,27.1,25.7,15.4,47.5,14.8,33.7,17.0
728,97.3,426.9,406.8,490.1,110.6,270.3,32.4,84.6,0.0,0.0,...,23.8,34.6,33.7,23.4,34.2,15.3,48.1,14.3,34.2,17.7
729,96.5,446.3,411.8,510.9,110.1,262.8,30.0,58.0,0.0,0.0,...,25.8,42.9,28.3,25.3,21.7,16.9,47.3,14.2,40.8,17.5


In [12]:
# * 예측 발전량과 부하량을 통해 전기 요금 산출
import pandas as pd
# res_solar_sum = pd.DataFrame(res_solar.sum(axis=1), columns=['Total Predicted Solar'])
# res_solar_sum

res_load_sum = pd.DataFrame(res_load.sum(axis=1), columns=['Total Predicted Solar'])
res_load_sum

,Total Predicted Solar
0,6889.0
1,6791.7
2,6679.7
3,6636.8
4,6378.9
5,6463.9
6,6583.7
7,6642.6
8,6410.9
9,6475.0
